In [3]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
from scipy.signal import convolve
import scipy
from joblib import Parallel, delayed
import time
from scipy import sparse
import json
from tqdm import tqdm, tqdm_notebook
import os
from antigenicWaveSimulationMethods import main as coEvoSimulation
from formulas import compute_shift
from coverage import elementwise_coverage
from fitness import fitness_spacers, norm_fitness, virus_growth
from altImmunity import immunity_loss_uniform, immunity_gain_from_kernel
from initMethods import init_exptail, init_full_kernel, init_quarter_kernel, init_guassian, init_cond
from supMethods import time_conv, write2json, minmax_norm
from mutation import mutation
from randomHGT import get_time_next_HGT, HGT_logistic_event

In [2]:
params = { #parameters relevant for the equations
        "Nh":                     1E5,
        "N0":                     1E9, #This Will be updated by self-consitent solution
        "R0":                      20, 
        "M":                       10, #Also L, total number of spacers
        "mu":                     100, #mutation rate
        "gamma_shape":             20,
        "Np":                     100, #Number of Cas Protein
        "dc":                      10, #Required number of complexes to activate defence
        "h":                       10, #coordination coeff
        "r":                     2000, #cross-reactivity kernel
        "beta":                     0,
        "rate_HGT":                 0,
        "HGT_bonus_acq_ratio":      0,
        "rate_recovery":            0,
        "HGT_type":                 0,
        "n_spacer":                 1,
        "A":                        1,
    }
sim_params = { #parameters relevant for the simulation (including Inital Valuess)
        "continue":                 False, #DO NOT CREATE ARBITRARY FOLDERS ONLY FOR TESTS
        "xdomain":                  10000,
        "dim":                          1,
        "dx":                           1,
        "tf":                        4000,
        "dt":                           1,
        "dt_exact_fitness":             1,
        "dt_snapshot":                  1,
        "initial_mean_n":           [0,0],
        "initial_mean_nh":          [0,0],
        "conv_size":                 4000,
        "num_threads":                 32,
        "foldername":  "../Data_Single_Test_Det_Fit",
        "seed":                         34,
        "hard_N0":                   False,
    }

In [3]:
params, sim_params = init_cond(params, sim_params)
foldername = sim_params["foldername"]

try:
    write2json(foldername, params, sim_params)
except FileNotFoundError:
    os.mkdir(foldername)
    write2json(foldername, params, sim_params)


In [4]:
st1 = time.time()
n = init_guassian(params["N"], sim_params, "n")
nh = init_exptail(params["Nh"]*params["M"], params, sim_params, "nh")
kernel_conv = init_quarter_kernel(params, sim_params)
kernel_immunity = init_quarter_kernel(params, sim_params, type="Boltzmann")
ed = time.time()
            
nh_total = params["Nh"]
n_total = params["N"]
uc = params["uc"]
sigma = params["sigma"]
M0 = params["M0"]
t = 0

with open(foldername+'/runtime_stats.txt','w') as file:
    file.write(f't: {t}| init_functions: {time_conv(ed-st1)}| Phage Population: {n_total:.4f}| Spacer Population: {nh_total:.4f}| Uc: {uc:.4f}| sigma: {sigma:.4f}| M0: {M0:.4f} \n')


In [ ]:
while(t < sim_params["tf"]):

    if t%sim_params["dt_snapshot"] == 0:
        sparse.save_npz(foldername+f"/sp_frame_n{t}",n.tocoo())
        sparse.save_npz(foldername+f"/sp_frame_nh{t}",nh.tocoo()) 

    str1:float = time.time()
    p = elementwise_coverage(nh, n, kernel_conv, params, sim_params)
    sparse.save_npz(foldername+f"/sp_frame_p{t}",p.tocoo())
    st2 = time.time()
    f = fitness_spacers(n, nh, p, params, sim_params)
    sparse.save_npz(foldername+f"/sp_frame_f{t}", f.tocoo())
    f = norm_fitness(f, n, params, sim_params) #renormalize f
    n = virus_growth(n, f, params, sim_params, True) #update
            
    if (np.sum(n) <= 0) or (np.sum(n) >= (1/2)*np.sum(nh)):
        with open(foldername+'/runtime_stats.txt','a') as file:
            outstring = f"DEAD at: {t}| N: {np.sum(n)}| Coverage: {time_conv(st2-st1)}| Growth: {time_conv(st3-st2)}| Mutation: {time_conv(st4-st3)}| Immunity: {time_conv(ed-st4)}| Shift Amount: {np.linalg.norm(shift_vector)} \n"
            file.write(outstring)

    st3 = time.time()
    n = mutation(n, params, sim_params)

    st4 = time.time()
    nh_prev = nh

    params, sim_params, num_to_add, num_to_remove = HGT_logistic_event(t, n, params, sim_params)
    nh_gain = immunity_gain_from_kernel(nh, n, kernel_immunity, params, sim_params, num_to_add) #update nh
    nh = immunity_loss_uniform(nh_gain, n, params, sim_params, num_to_remove)
            
    diff_of_acquisition = num_to_add-num_to_remove
    shift_vector = compute_shift(nh, nh_prev, "max")
    ed = time.time()

    with open(foldername+'/runtime_stats.txt','a') as file:
        M = params["M"]
        outstring = f"t: {t}| N: {np.sum(n)}| Coverage: {time_conv(st2-st1)}| Growth: {time_conv(st3-st2)}| Mutation: {time_conv(st4-st3)}| Immunity: {time_conv(ed-st4)}| M: {M:.4f}| Net_Acq_Diff: {diff_of_acquisition:.4f}| Shift Amount: {np.linalg.norm(shift_vector):.4f} \n"
        file.write(outstring)

    t += sim_params["dt"]

In [6]:
from initMethods import init_dict_kernel

kernel_dict = init_dict_kernel(params, sim_params, type = "coverage", exponent = 1)

In [10]:
import os
import numpy as np

# Set the number of threads for OpenBLAS/MKL
os.environ["OMP_NUM_THREADS"] = "32"  # Set this to the desired number of threads
os.environ["OPENBLAS_NUM_THREADS"] = "32"
os.environ["MKL_NUM_THREADS"] = "32"
os.environ["VECLIB_MAXIMUM_THREADS"] = "32"
os.environ["NUMEXPR_NUM_THREADS"] = "32"

In [15]:
from copy import deepcopy
from scipy.spatial.distance import cdist

def lookup_value(val):
    val = float(val)
    return kernel_dict.get(val, 0.)

def convolve_subset(A, nonzero_values):
    print("Fuck this is happening")
    res = np.zeros(len_ind_n)

    # dist = cdist(A, B)

    for i in range(len_ind_n): #go through indexes of n
        dist = cdist(A, B[i, :].reshape(1,2))
        res[i] = np.dot(np.vectorize(lookup_value)(dist).squeeze(), nonzero_values)
        # res[i] = np.dot(np.vectorize(lookup_value)(dist[:, 0]).squeeze(), nonzero_values)
        # res[i] = np.dot(dist[:, 0], nonzero_values)
        # dist = dist[:, 1:]

Nh = params["Nh"]
M = params["M"]
# num_threads = 1
num_threads = sim_params["num_threads"]

x_ind_nh, y_ind_nh = nh.nonzero()
x_ind_n, y_ind_n = n.nonzero()

A = np.array([x_ind_nh, y_ind_nh]).transpose()
A_sets = np.array_split(A, num_threads, axis = 0)
B = np.array([x_ind_n, y_ind_n]).transpose()
len_ind_n = len(x_ind_n)

input_h = np.divide(nh, Nh*M)
nonzero_values = np.array(input_h[x_ind_nh, y_ind_nh].toarray()).squeeze()

x_nh_sets = np.array_split(x_ind_nh, num_threads)
y_nh_sets = np.array_split(y_ind_nh, num_threads)
# result_values = convolve_subset(A, input_h)
# kernel_dict_copies = [deepcopy(kernel_dict) for _ in range(num_threads)]



# nonzero_values_sets = np.array_split(nonzero_values, num_threads)
# results = Parallel(n_jobs=num_threads, backend="loky")(delayed(convolve_subset)
#         (A, nonzero_values)
#             for A, nonzero_values
#                 in zip(A_sets, nonzero_values_sets))

# result_values = sum_parallel(results, num_threads)
# res = scipy.sparse.dok_matrix(n.shape, dtype=float)
# res[x_ind_n, y_ind_n] = result_values

In [ ]:
from copy import deepcopy
from scipy.spatial.distance import cdist
from multiprocessing import Manager, Pool

# Function to perform the dictionary lookup
def lookup_value(shared_dict, key):
    return shared_dict.get(key, 0)

def parallel_lookup(shared_dict, keys_to_lookup):
    # Create a pool of workers
    with Pool(processes= num_threads) as pool:
        # Perform parallel lookup
        results = pool.starmap(lookup_value, [(shared_dict, key) for key in keys_to_lookup])
    return results

Nh = params["Nh"]
M = params["M"]
# num_threads = 1
num_threads = sim_params["num_threads"]

x_ind_nh, y_ind_nh = nh.nonzero()
x_ind_n, y_ind_n = n.nonzero()

len_ind_n = len(x_ind_n)

input_h = np.divide(nh, Nh*M)

x_nh_sets = np.array_split(x_ind_nh, num_threads)
y_nh_sets = np.array_split(y_ind_nh, num_threads)

nonzero_values = np.array(input_h[x_ind_nh, y_ind_nh].toarray()).squeeze()
nonzero_values_sets = np.array_split(nonzero_values, num_threads)


results = parallel_lookup(kernel_dict, )


In [25]:
foldername = "../Data/test13"
with open(foldername + "/params.json") as json_file:
    params = json.load(json_file)
with open(foldername + "/sim_params.json") as json_file:
    sim_params = json.load(json_file)

i = 999
kernel_quarter = init_quarter_kernel(params, sim_params)

n = sparse.load_npz(foldername+f"/sp_frame_n{i}.npz").todok()
nh = sparse.load_npz(foldername+f"/sp_frame_nh{i}.npz").todok()